In [1]:
!pip install pydot

In [2]:
import pandas as pd
import random
#import torch
#from transformers import AutoTokenizer, AutoModel
#import re
#import string
import numpy as np
import pydot
#from nltk.corpus import stopwords 
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import LancasterStemmer
#from simpletransformers.classification import MultiLabelClassificationModel
#import logging
#import custom_sentence_tokenizer
import matplotlib.pyplot as plt
#from scipy import stats
#from ast import literal_eval
import pickle
#from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
import keras
import gc
from keras.models import model_from_json

Using TensorFlow backend.


In [3]:
#from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Reshape
from keras.layers import GlobalMaxPooling1D,Conv2D, Conv1D, AveragePooling2D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import concatenate
#from sklearn.decomposition import PCA
import _pickle as cPickle
import io
import time
import sys
import boto3
from boto3.s3.transfer import TransferConfig
import tensorflow as tf

## 1. Import Embeddings (X) and Labels (Y)

In [4]:
now = time.time()
infile = None
infile = open("embeddings_7",'rb')
embeddings = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)
gc.collect()

44.7971134185791


0

In [5]:
now = time.time()
infile = None
infile = open("embeddings_evaluation",'rb')
embeddings_evaluation = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)
gc.collect()

43.72156071662903


0

In [6]:
embeddings.update(embeddings_evaluation)
pickle.dump( embeddings, open( "/home/ec2-user/SageMaker/embeddings.p", "wb" ) )

In [ ]:
now = time.time()
infile = None
infile = open("embeddings.p",'rb')
embeddings = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)
gc.collect()

In [7]:
embeddings_np_complete = np.array(list(embeddings.items()))
del(embeddings)

## 2. Load chapter model and helper functions

In [8]:
#Load chapeter model weights
chapter_model = None
json_file = open('model_num_cnn2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
chapter_model = model_from_json(loaded_model_json)
opt = keras.optimizers.Adam(lr=0.001)
# load weights into new model
chapter_model.load_weights("model_num_cnn2.h5")

In [29]:
def padding(embeddings):
    max_sentence = 20
    padding = max_sentence - len(embeddings)
    if padding > 0:
        padding_shape = (padding, 256, 768)
        pad = np.zeros(padding*256*768).reshape(padding_shape)
        return np.append(embeddings, pad, axis = 0).astype('float16')
    else:
        return embeddings
    
def split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat):
    HADM_ID_DF = None
    loaded_model = None
    HADM_ID_DF =  pd.DataFrame(data=HADM_ID_LIST, columns=['HADM_ID'])
    CHAPTER_PIVOT_DF_COPY = HADM_ID_DF.set_index('HADM_ID')\
                    .join(CHAPTER_PIVOT_DF, how='left').copy(deep=False)
    CHAPTER_PIVOT_NP = np.array(CHAPTER_PIVOT_DF_COPY)
    return train_test_split(embeddings_concat, CHAPTER_PIVOT_NP, test_size=0.2)

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def get_model(label_len):
    #define model - STATIC LAYERS
    model = Sequential()
    for layer in chapter_model.layers[:4]:
        layer.trainable = False
        model.add(layer)
    
    #LEARNING LAYERS
    if label_len <=4:
        print('model 1')
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(label_len, activation='sigmoid'))     
    elif label_len >4:
        print('model 2')
        #model.add(Conv1D(label_len, 4))
        #model.add(AveragePooling1D(pool_size=2))
        #model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(64, activation='relu'))
        #model.add(Dropout(0.3))
        model.add(Dense(label_len, activation='sigmoid'))   

    return model

## 3. Loop on individual therapy area (bag of models)

In [ ]:
score_list = []
history_list = []
file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','280_289.csv','290_319.csv','320_389.csv',
                  '390_459.csv', '460_519.csv','520_579.csv','580_629.csv']
#,'460_519.csv','520_579.csv','580_629.csv','710-739.csv','780-799.csv','800-999.csv','E_V.csv']
#'520_579.csv','580_629.csv','630_679__740_759__760_779.csv','710-739.csv','780-799.csv','800-999.csv','E_V.csv']
1. #file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','800-999.csv']
#file_name_LIST = ['240_279_1.csv']
#file_name_LIST = ['280_289.csv','290_319.csv','320_389.csv','390_459.csv','460_519.csv','520_579.csv','580_629.csv']

for file_name in file_name_LIST:
    print('\n\n============Started '+file_name+'============')
    CHAPTER_PIVOT_DF = pd.read_csv('Layer-2/'+file_name, sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
    HADM_ID_LIST = list(CHAPTER_PIVOT_DF.index)
    embeddings_np = embeddings_np_complete[np.isin(embeddings_np_complete[:,0], HADM_ID_LIST)]
    print(file_name+' orginal shape: '+str(embeddings_np.shape))
    HADM_ID_LIST = embeddings_np[:,0].copy()
    embeddings = embeddings_np[:,1].copy()
    
    #Zero Pad embeddings
    embeddings_padded = [padding(embeddings_ITEM) for embeddings_ITEM in embeddings]
    del(embeddings)
    gc.collect()
    print(file_name+' Padding complete')

    #Reshape embeddings
    #embeddings_concat = np.array([np.concatenate(i) for i in embeddings_padded])
    embeddings_concat = np.array(embeddings_padded).reshape(len(embeddings_padded), 20*256, 768)
    print(file_name+' modified shape: '+str(embeddings_concat.shape))
    del(embeddings_padded)
    gc.collect()
    
    X_train2, X_test2, y_train2, y_test2 = split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat)
    print('Train and Test Shapre: '+str(X_train2.shape)+', '+str(X_test2.shape)+', '+str(y_train2.shape)+', '+str(y_test2.shape))
    X_val = X_test2[:int(0.25*len(X_test2))]
    y_val = y_test2[:int(0.25*len(y_test2))]
    del(HADM_ID_LIST)
    del(CHAPTER_PIVOT_DF)
    del(embeddings_concat)
    gc.collect()
    
    #call model
    label_len = y_train2.shape[1]
    model = get_model(label_len)
    opt = keras.optimizers.Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
    history = model.fit(X_train2, y_train2, epochs=4,batch_size=32, verbose=1, validation_data = (X_val,y_val))
    
    #evaluate model
    print('Model evaluation '+file_name)
    score = model.evaluate(X_test2, y_test2, verbose=1)
    print(score)
    
    #save model evaluation and history metrics
    score_list.append(score)
    history_list.append(history.history)
    
    #save model
    model_json = model.to_json()
    with open("/home/ec2-user/SageMaker/Models/model_"+file_name+".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("/home/ec2-user/SageMaker/Models/model_"+file_name+".h5")
    print('============completed '+file_name+'============')



============Started 001_139.csv============
001_139.csv orginal shape: (5135, 2)
001_139.csv Padding complete
001_139.csv modified shape: (5135, 5120, 768)
Train and Test Shapre: (4108, 5120, 768), (1027, 5120, 768), (4108, 4), (1027, 4)
model 1
Train on 4108 samples, validate on 256 samples
Epoch 1/5
4108/4108 [==============================] - 161s 39ms/step - loss: 0.5635 - get_f1: 0.5742 - accuracy: 0.7436 - val_loss: 0.4979 - val_get_f1: 0.5951 - val_accuracy: 0.7764
Epoch 2/5
4108/4108 [==============================] - 160s 39ms/step - loss: 0.4808 - get_f1: 0.6283 - accuracy: 0.7842 - val_loss: 0.4861 - val_get_f1: 0.6120 - val_accuracy: 0.7803
Epoch 3/5
4108/4108 [==============================] - 161s 39ms/step - loss: 0.4351 - get_f1: 0.6764 - accuracy: 0.8095 - val_loss: 0.4902 - val_get_f1: 0.6212 - val_accuracy: 0.7881
Epoch 4/5
4108/4108 [==============================] - 160s 39ms/step - loss: 0.3895 - get_f1: 0.7156 - accuracy: 0.8323 - val_loss: 0.5012 - val_get_f1:

In [ ]:
#'001_139.csv','140_239.csv','240_279.csv','800-999.csv'
# Run 800-999 and 240-279 with more epochs: 6 and 10
pickle.dump( score_list, open( "/home/ec2-user/SageMaker/score_list_1.p", "wb" ) )
pickle.dump( history_list, open( "/home/ec2-user/SageMaker/history_list_1.p", "wb" ) )

In [ ]:
score_list = []
history_list = []
file_name_LIST = ['adverse_effect.csv']
#,'460_519.csv','520_579.csv','580_629.csv','710-739.csv','780-799.csv','800-999.csv','E_V.csv']
#'520_579.csv','580_629.csv','630_679__740_759__760_779.csv','710-739.csv','780-799.csv','800-999.csv','E_V.csv']
1. #file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','800-999.csv']
#file_name_LIST = ['240_279_1.csv']
#file_name_LIST = ['280_289.csv','290_319.csv','320_389.csv','390_459.csv','460_519.csv','520_579.csv','580_629.csv']

for file_name in file_name_LIST:
    print('\n\n============Started '+file_name+'============')
    CHAPTER_PIVOT_DF = pd.read_csv('Layer-2/'+file_name, sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
    HADM_ID_LIST = list(CHAPTER_PIVOT_DF.index)
    embeddings_np = embeddings_np_complete[np.isin(embeddings_np_complete[:,0], HADM_ID_LIST)]
    print(file_name+' orginal shape: '+str(embeddings_np.shape))
    HADM_ID_LIST = embeddings_np[:,0].copy()
    embeddings = embeddings_np[:,1].copy()
    
    #Zero Pad embeddings
    embeddings_padded = [padding(embeddings_ITEM) for embeddings_ITEM in embeddings]
    del(embeddings)
    gc.collect()
    print(file_name+' Padding complete')

    #Reshape embeddings
    #embeddings_concat = np.array([np.concatenate(i) for i in embeddings_padded])
    embeddings_concat = np.array(embeddings_padded).reshape(len(embeddings_padded), 20*256, 768)
    print(file_name+' modified shape: '+str(embeddings_concat.shape))
    del(embeddings_padded)
    gc.collect()
    
    X_train2, X_test2, y_train2, y_test2 = split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat)
    print('Train and Test Shapre: '+str(X_train2.shape)+', '+str(X_test2.shape)+', '+str(y_train2.shape)+', '+str(y_test2.shape))
    X_val = X_test2[:int(0.25*len(X_test2))]
    y_val = y_test2[:int(0.25*len(y_test2))]
    del(HADM_ID_LIST)
    del(CHAPTER_PIVOT_DF)
    del(embeddings_concat)
    gc.collect()
    
    #call model
    label_len = y_train2.shape[1]
    model = get_model(label_len)
    opt = keras.optimizers.Adam(lr=0.00001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
    history = model.fit(X_train2, y_train2, epochs=5,batch_size=32, verbose=1, 
                        validation_data = (X_val,y_val))
    
    #evaluate model
    print('Model evaluation '+file_name)
    score = model.evaluate(X_test2, y_test2, verbose=1)
    print(score)
    
    #save model evaluation and history metrics
    score_list.append(score)
    history_list.append(history.history)
    
    #save model
    model_json = model.to_json()
    with open("/home/ec2-user/SageMaker/Models/model_"+file_name+".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("/home/ec2-user/SageMaker/Models/model_"+file_name+".h5")
    print('============completed '+file_name+'============')

In [37]:
def get_model(label_len):
    #define model - STATIC LAYERS
    model = Sequential()
    for layer in chapter_model.layers[:4]:
        layer.trainable = False
        model.add(layer)
    
    #LEARNING LAYERS
    if label_len <4:
        print('model 1')
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(label_len, activation='sigmoid'))     
    elif label_len >=4:
        print('model 2')
        #model.add(Conv1D(label_len, 4))
        #model.add(AveragePooling1D(pool_size=2))
        #model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(64, activation='relu'))
        #model.add(Dropout(0.3))
        model.add(Dense(label_len, activation='sigmoid'))   

    return model

In [ ]:
label_len = y_train2.shape[1]
model = get_model(label_len)
opt = keras.optimizers.Adam(lr=0.00001)
model.compile(loss=f1_loss, optimizer=opt, metrics=[get_f1, 'accuracy'])
history = model.fit(X_train2, y_train2, epochs=5,batch_size=32, verbose=1, 
                        validation_data = (X_val,y_val))
    
#evaluate model
print('Model evaluation '+file_name)
score = model.evaluate(X_test2, y_test2, verbose=1)
print(score)

In [34]:
import tensorflow as tf
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [44]:
def get_model(label_len):
    #define model - STATIC LAYERS
    model = Sequential()
    for layer in chapter_model.layers[:4]:
        layer.trainable = False
        model.add(layer)
    return model

label_len = y_train2.shape[1]
model = get_model(label_len)
opt = keras.optimizers.Adam(lr=0.00001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
flat_train = model.predict(X_train2, verbose = 1)

5071/5071 [==============================] - 183s 36ms/step


In [45]:
type(flat_train)

numpy.ndarray

In [47]:
flat_train.shape

(5071, 20448)

In [50]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 10, random_state = 42, verbose = 1)
rf.fit(flat_train, y_train2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  8.5min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=42, verbose=1, warm_start=False)

In [51]:
predictions = rf.predict(flat_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [52]:
predictions.shape

(5071, 4)

In [55]:
predictions[predictions>0.5] = 1
predictions[predictions<=0.5] = 0

In [57]:
import sklearn
sklearn.metrics.f1_score(y_train2, predictions, average='weighted')

0.97070317724093

In [59]:
label_len = y_test2.shape[1]
model = get_model(label_len)
opt = keras.optimizers.Adam(lr=0.00001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
flat_test = model.predict(X_test2, verbose = 1)

1268/1268 [==============================] - 46s 37ms/step


In [61]:
predictions = rf.predict(flat_test)
predictions[predictions>0.5] = 1
predictions[predictions<=0.5] = 0
sklearn.metrics.f1_score(y_test2, predictions, average='weighted')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


0.2815310105517057